In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import statsmodels.api as sm
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
import itertools as iter
import warnings
warnings.filterwarnings("ignore")

In [ ]:
Dataset=pd.read_csv('Dataset.csv')
X=Dataset[['AT','V','AP','RH']]
y=Dataset[['PE']]
X_dash,X_test,y_dash,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

# Outliers: assume that we will define an outlier as a sample that differes from the mean by a significant amount (let's say 2.5 standard deviations)
feature=['AT','V','AP','RH']
for j in range(4):
    number_of_outliers=0
    for i in Dataset[feature[j]]:
        if i>=Dataset[feature[j]].mean()+3*Dataset[feature[j]].std()  or  i<=Dataset[feature[j]].mean()-3*Dataset[feature[j]].std():
            number_of_outliers=number_of_outliers+1
    print(feature[j],'has',number_of_outliers,'outliers')
sns.boxplot(x=X_dash['AT'])
plt.show()
sns.boxplot(x=X_dash['V'])
plt.show()
sns.boxplot(x=X_dash['AP'])
plt.show()
sns.boxplot(x=X_dash['RH'])
plt.show()


#study the correlation between variables and the prediction power of the feeatures
corr = X_dash[['AT','V','AP','RH']].join(y_dash['PE']).corr()
plt.figure(figsize=(15,10))
sns.heatmap(corr, annot=True, annot_kws={'size':15})
plt.show()


model=DecisionTreeRegressor()
model.fit(X_dash[['AT','V','AP','RH']],y_dash)
features=pd.Series(model.feature_importances_,X_dash[['AT','V','AP','RH']].columns).sort_values(ascending=False)
features.plot(kind='bar',title='Feature Importance')
plt.show()





#train and validation sets splits
X_train,X_val,y_train,y_val=train_test_split(X_dash,y_dash,train_size=0.75,random_state=0)

def pset(lst):
    comb = (iter.combinations(lst, l) for l in range(len(lst) + 1))
    res=list(iter.chain.from_iterable(comb))
    return res


#best subset selection
c=pset(X_dash[['AT','V','AP','RH']])
valadation_scores={}
for i in range(1,16,1):
    model=LinearRegression()
    model.fit(X_train[list(c[i])],y_train)
    y_train_pred=model.predict(X_train[list(c[i])])
    y_val_pred=model.predict(X_val[list(c[i])])
    valadation_scores.update({c[i]:mean_squared_error(y_val,y_val_pred)})

best_val_score=min(valadation_scores, key=valadation_scores.get)
print(valadation_scores)


final_scores={}
for i in range(1,16,1):
    model=LinearRegression()
        
    scores=cross_val_score(model,X_dash[list(c[i])], y_dash,cv=5, scoring='neg_mean_squared_error')
    final_scores.update({c[i]:scores.mean()})
    
best_feature_set=max(final_scores, key=final_scores.get)
print(final_scores)
print('best featrue subset are:',best_feature_set)




#study the interaction terms
#add the interation terms to our feature sets
X_dash_=pd.DataFrame([X_dash['AT'],X_dash['V'],X_dash['AP'],X_dash['RH'],X_dash['AT']*X_dash['V'],X_dash['AT']*X_dash['AP'],X_dash['AT']*X_dash['RH'],X_dash['V']*X_dash['AP'],X_dash['V']*X_dash['RH'],X_dash['AP']*X_dash['RH']])
X_dash_=X_dash_.T
X_dash_.columns=['AT','V','AP','RH','AT*V','AT*AP','AT*RH','V*AP','V*RH','AP*RH']

X_test_=pd.DataFrame([X_test['AT'],X_test['V'],X_test['AP'],X_test['RH'],X_test['AT']*X_test['V'],X_test['AT']*X_test['AP'],X_test['AT']*X_test['RH'],X_test['V']*X_test['AP'],X_test['V']*X_test['RH'],X_test['AP']*X_test['RH']])
X_test_=X_test_.T
X_test_.columns=['AT','V','AP','RH','AT*V','AT*AP','AT*RH','V*AP','V*RH','AP*RH']

#fit a linear regression model and obseve the p-value of the interaction terms to determine their significance
X_dash_withconsnt= sm.add_constant(X_dash_[['AT','V','AP','RH','AT*V','AT*AP','AT*RH','V*AP','V*RH','AP*RH']])
model = sm.OLS(y_dash, X_dash_withconsnt)
model = model.fit()
print(model.summary())



X_dash_withconsnt= sm.add_constant(X_dash)
model = sm.OLS(y_dash, X_dash_withconsnt)
model = model.fit()
print(model.summary())


model=DecisionTreeRegressor()
model.fit(X_dash_,y_dash)
features=pd.Series(model.feature_importances_,X_dash_.columns).sort_values(ascending=False)
features.plot(kind='bar',title='Feature Importance')
plt.show()

#now we will find the best set of interaction terms using a validation set.

X_train,X_val,y_train,y_val=train_test_split(X_dash_,y_dash,train_size=0.75,random_state=0)

c=pset(X_dash_[['AT*V','AT*AP','AT*RH','V*AP','V*RH','AP*RH']])


valadation_scores={}
training_scores={}
for i in range(1,64,1):
    model=LinearRegression()
    model.fit(X_train[['AT','V','AP','RH']].join(X_train[list(c[i])]),y_train)
    y_train_pred=model.predict(X_train[['AT','V','AP','RH']].join(X_train[list(c[i])]))
    y_val_pred=model.predict(X_val[['AT','V','AP','RH']].join(X_val[list(c[i])]))
    training_scores.update({c[i]:mean_squared_error(y_train,y_train_pred)})
    valadation_scores.update({c[i]:mean_squared_error(y_val,y_val_pred)})

best_val_score=min(valadation_scores, key=valadation_scores.get)

best_training_scores={}
for j in range(1,7,1):
    temp={}
    for i in training_scores.keys():
        if len(i)==j:
            temp.update({i:training_scores[i]})
    best_training_scores.update({j:min(temp, key=temp.get)})

print('best fitted model for each d',best_training_scores)

best_validation_scores={}
for j in best_training_scores.values():
    best_validation_scores.update({j:valadation_scores[j]})
    
best_feature_set=min(best_validation_scores, key=valadation_scores.get)
print('validation error for each best fitted model with d features',best_validation_scores)



# now we will use these features to learn multiple models that we learned from EE 660
X_train=X_train[['AT','V','AP','RH']].join(X_train[list(best_feature_set)])
X_val=X_val[['AT','V','AP','RH']].join(X_val[list(best_feature_set)])
X_test=X_test_[['AT','V','AP','RH']].join(X_test_[list(best_feature_set)])
X_dash=X_dash_[['AT','V','AP','RH']].join(X_dash_[list(best_feature_set)])

In [ ]:
#lasso and Ridge
#This is the code for the standardized case using validation set
scaler=StandardScaler()
scaler.fit(X_train)
X_train_std=scaler.transform(X_train)
X_val_std=scaler.transform(X_val)
X_test_std=scaler.transform(X_test)

valadation_scores={}
for i in np.linspace(-10,10,21):
    model=Lasso(alpha=2**i)
    model.fit(X_train_std,y_train)
    y_val_pred=model.predict(X_val_std)
    valadation_scores.update({2**i:mean_squared_error(y_val,y_val_pred)})

best_lambda_laaso=min(valadation_scores, key=valadation_scores.get)


model=Lasso(alpha=best_lambda_laaso)
model.fit(X_train_std,y_train)
y_train_pred=model.predict(X_train_std)
y_val_pred=model.predict(X_val_std)

print('LASSOTrain MSE is',mean_squared_error(y_train,y_train_pred))
print('LASSOValadation set MSE is:',mean_squared_error(y_val,y_val_pred))
print('best lambda Lasso is:',best_lambda_laaso)
plt.plot(np.linspace(-10,10,21),valadation_scores.values())
plt.xlabel('regularization parameter')
plt.ylabel('validation error')
plt.show()


valadation_scores={}
for i in np.linspace(-10,10,21):
    model=Ridge(alpha=2**i)
    model.fit(X_train_std,y_train)
    y_val_pred=model.predict(X_val_std)
    valadation_scores.update({2**i:mean_squared_error(y_val,y_val_pred)})

best_lambda_Ridge=min(valadation_scores, key=valadation_scores.get)


model=Ridge(alpha=best_lambda_Ridge)
model.fit(X_train_std,y_train)
y_train_pred=model.predict(X_train_std)
y_val_pred=model.predict(X_val_std)

print('RidgeTrain MSE is',mean_squared_error(y_train,y_train_pred))
print('RidgeValadation set MSE is:',mean_squared_error(y_val,y_val_pred))
print('best lambda ridge is:',best_lambda_Ridge)
plt.plot(np.linspace(-10,10,21),valadation_scores.values())
plt.xlabel('regularization parameter')
plt.ylabel('validation error')
plt.show()

#study the variabity of diffrent training size for Lasso and Ridge
best_lambdas_laaso=[]
validation_scores=[]
N=[0.1,0.2 ,0.3,0.4,0.5, 0.6,0.7,0.8, 0.9,0.999]
for j in N:
    valadation_scores={}
    
    for i in np.linspace(-10,10,21):
        model=Lasso(alpha=2**i)
        X_train_bagged,X_throw,y_train_bagged,y_throw=train_test_split(X_train_std,y_train,train_size=j ,random_state=0)
        model.fit(X_train_bagged,y_train_bagged)
        y_val_pred=model.predict(X_val_std)
        valadation_scores.update({2**i:mean_squared_error(y_val,y_val_pred)})
    
    best_lambda_laaso=min(valadation_scores, key=valadation_scores.get)
    best_lambdas_laaso.append(best_lambda_laaso)


    model=Lasso(alpha=best_lambda_laaso)
    model.fit(X_train_bagged,y_train_bagged)
    y_train_pred=model.predict(X_train_bagged)
    validation_scores.append(mean_squared_error(y_val,model.predict(X_val_std)))
    
    
plt.plot(N,best_lambdas_laaso);plt.xlabel('Training Size');plt.ylabel('best_lambdas_laaso')
plt.show()
plt.plot(N,validation_scores);plt.xlabel('Training Size');plt.ylabel('validation error')
plt.show()


best_lambdas_Ridge=[]
validation_scores=[]
N=[0.1,0.2 ,0.3,0.4,0.5, 0.6,0.7,0.8, 0.9,0.999]
for j in N:
    valadation_scores={}
    
    for i in np.linspace(-10,10,21):
        model=Ridge(alpha=2**i)
        X_train_bagged,X_throw,y_train_bagged,y_throw=train_test_split(X_train_std,y_train,train_size=j ,random_state=0)
        model.fit(X_train_bagged,y_train_bagged)
        y_val_pred=model.predict(X_val_std)
        valadation_scores.update({2**i:mean_squared_error(y_val,y_val_pred)})
    
    best_lambda_Ridge=min(valadation_scores, key=valadation_scores.get)
    best_lambdas_Ridge.append(best_lambda_Ridge)


    model=Ridge(alpha=best_lambda_Ridge)
    model.fit(X_train_bagged,y_train_bagged)
    y_train_pred=model.predict(X_train_bagged)
    validation_scores.append(mean_squared_error(y_val,model.predict(X_val_std)))
    
plt.plot(N,best_lambdas_Ridge);plt.xlabel('Training Size');plt.ylabel('best_lambdas_Ridge')
plt.show()
plt.plot(N,validation_scores);plt.xlabel('Training Size');plt.ylabel('validation error')
plt.show()



#study the robusness to the noise for Lasso and Ridge
best_lambda_Lasso_mean=[]
for it in np.arange(1,9,1):
    best_lambdas_Lasso=[]
    validation_scores=[]
    noise_std=np.arange(0.01,0.09,0.01)
    for j in noise_std:
        valadation_scores={}
    
        for i in np.linspace(-10,10,21):
            model=Lasso(alpha=2**i)
            X_train_std_noisy=X_train_std+np.random.normal(0,j,[5740,7])
            model.fit(X_train_std_noisy,y_train)
            y_val_pred=model.predict(X_val_std)
            valadation_scores.update({2**i:mean_squared_error(y_val,y_val_pred)})
    
        best_lambda_Lasso=min(valadation_scores, key=valadation_scores.get)
        best_lambdas_Lasso.append(best_lambda_Lasso)


        model=Lasso(alpha=best_lambda_Lasso)
        model.fit(X_train_std_noisy,y_train)
        y_train_pred=model.predict(X_train_std_noisy)
        validation_scores.append(mean_squared_error(y_val,model.predict(X_val_std)))
        
    
    best_lambda_Lasso_mean.append(np.mean(best_lambdas_Lasso))

plt.plot(noise_std,best_lambda_Lasso_mean,c='r');plt.xlabel('noise_std');plt.ylabel('best_lambdas_Lasso')
plt.show()
plt.plot(noise_std,validation_scores,c='r');plt.xlabel('noise_std');plt.ylabel('validation error')
plt.show()

best_lambda_Ridge_mean=[]
for it in np.arange(1,9,1):
    best_lambdas_Ridge=[]
    validation_scores=[]
    noise_std=np.arange(0.01,0.09,0.01)
    for j in noise_std:
        valadation_scores={}
    
        for i in np.linspace(-10,10,21):
            model=Ridge(alpha=2**i)
            X_train_std_noisy=X_train_std+np.random.normal(0,j,[5740,7])
            model.fit(X_train_std_noisy,y_train)
            y_val_pred=model.predict(X_val_std)
            valadation_scores.update({2**i:mean_squared_error(y_val,y_val_pred)})
    
        best_lambda_Ridge=min(valadation_scores, key=valadation_scores.get)
        best_lambdas_Ridge.append(best_lambda_Ridge)


        model=Ridge(alpha=best_lambda_Ridge)
        model.fit(X_train_std_noisy,y_train)
        y_train_pred=model.predict(X_train_std_noisy)
        validation_scores.append(mean_squared_error(y_val,model.predict(X_val_std)))
    
    best_lambda_Ridge_mean.append(np.mean(best_lambdas_Ridge))

plt.plot(noise_std,best_lambda_Ridge_mean,c='r');plt.xlabel('noise_std');plt.ylabel('best_lambdas_Ridge')
plt.show()
plt.plot(noise_std,validation_scores,c='r');plt.xlabel('noise_std');plt.ylabel('validation error')
plt.show()

In [ ]:
#CART
valadation_scores={}

for i in range(20):
    model=DecisionTreeRegressor(max_depth=i+1,criterion='mse',max_features=None,random_state=0)
    model.fit(X_train,y_train)
    y_val_pred=model.predict(X_val)
    valadation_scores.update({i+1:mean_squared_error(y_val,y_val_pred)})
    
best_depth=min(valadation_scores, key=valadation_scores.get)


model=DecisionTreeRegressor(max_depth=best_depth,criterion='mse',max_features=None,
                            random_state=0)
model.fit(X_train,y_train)
y_train_pred=model.predict(X_train)
y_val_pred=model.predict(X_val)
y_test_pred=model.predict(X_test)
print('CART Train MSE is',mean_squared_error(y_train,y_train_pred))
print('CART Valadation set MSE is:',mean_squared_error(y_val,y_val_pred))
plt.plot(valadation_scores.keys(),valadation_scores.values())
plt.xlabel('Depth of The Tree')
plt.ylabel('Validation Error')
plt.show()
print('best depth is:',best_depth)

#study the variabity of diffrent training size for CART
best_depths=[]
validation_scores=[]
N=np.arange(0.01,0.99,0.01)
for j in N:
    valadation_scores={}
    
    for i in range(20):
        model=DecisionTreeRegressor(max_depth=i+1,criterion='mse',max_features=None,random_state=0)
        X_train_bagged,X_throw,y_train_bagged,y_throw=train_test_split(X_train,y_train,train_size=j ,random_state=0)
        model.fit(X_train_bagged,y_train_bagged)
        y_val_pred=model.predict(X_val)
        valadation_scores.update({i+1:mean_squared_error(y_val,y_val_pred)})
    
    best_depth=min(valadation_scores, key=valadation_scores.get)
    best_depths.append(best_depth)


    model=DecisionTreeRegressor(max_depth=best_depth,criterion='mse',max_features=None,random_state=0)
    model.fit(X_train_bagged,y_train_bagged)
    y_train_pred=model.predict(X_train_bagged)
    validation_scores.append(mean_squared_error(y_val,model.predict(X_val)))
    


plt.plot(N,best_depths);plt.xlabel('Training Size');plt.ylabel('Best Depth')
plt.show()
plt.plot(N,validation_scores);plt.xlabel('Training Size');plt.ylabel('validation error')
plt.show()

#study the robusness to the noise for CART
best_depths=[]
validation_scores=[]
noise_std=np.arange(0.01,0.1,0.01)
for j in noise_std:
    valadation_scores={}
    
    for i in range(20):
        model=DecisionTreeRegressor(max_depth=i+1,criterion='mse',max_features=None,random_state=0)
        X_train_std_noisy=X_train_std+np.random.normal(0,j,[5740,7])
        model.fit(X_train_std_noisy,y_train)
        y_val_pred=model.predict(X_val_std)
        valadation_scores.update({i+1:mean_squared_error(y_val,y_val_pred)})
    
    best_depth=min(valadation_scores, key=valadation_scores.get)
    best_depths.append(best_depth)


    model=DecisionTreeRegressor(max_depth=best_depth,criterion='mse',max_features=None,random_state=0)
    model.fit(X_train_std_noisy,y_train)
    y_train_pred=model.predict(X_train_std_noisy)
    validation_scores.append(mean_squared_error(y_val,model.predict(X_val_std)))
    


plt.plot(noise_std,best_depths,c='r');plt.xlabel('noise_std');plt.ylabel('Best Depth')
plt.show()
plt.plot(noise_std,validation_scores,c='r');plt.xlabel('noise_std');plt.ylabel('validation error')
plt.show()

In [ ]:
#Random Forest
valadation_scores={}
for i in range(2,100):
    model=RandomForestRegressor(n_estimators=i,criterion='mse',max_depth=best_depth,
                                max_features=None,random_state=0)
    model.fit(X_train,y_train)
    y_val_pred=model.predict(X_val)
    valadation_scores.update({i:mean_squared_error(y_val,y_val_pred)})
    
best_tree_num=min(valadation_scores, key=valadation_scores.get)

model=RandomForestRegressor(n_estimators=best_tree_num,criterion='mse',max_depth=best_depth,
                            max_features=None,random_state=0)
model.fit(X_train,y_train)
y_train_pred=model.predict(X_train)
y_val_pred=model.predict(X_val)
y_test_pred=model.predict(X_test)
print('RF Train MSE is',mean_squared_error(y_train,y_train_pred))
print('RF Valadation set MSE is:',mean_squared_error(y_val,y_val_pred))
print('best tree numberis:',best_tree_num)
plt.plot(valadation_scores.keys(),valadation_scores.values())
plt.xlabel('Number of Trees')
plt.ylabel('Validation Error')
plt.show()

#study the variabity of diffrent training size for Random Forest
def find_vectors_and_plots(B,bagg_size,max_depthh,max_featuress):
    train_mean_error_rate=[]
    train_std_error_rate=[]
    validation_mean_error_rate=[]
    validation_std_error_rate=[]


    for i in range(B):
        training_error_rate=[]
        validation_error_rate=[]
    
        for j in range(10):
            X_train_bagged,X_test_bagged,y_train_bagged,ytest_bagged=train_test_split(X_train,y_train,
            train_size=bagg_size)
            model=RandomForestRegressor(n_estimators=i+1,criterion='mse',max_depth=max_depthh,bootstrap=True,max_features=max_featuress)
            
            model.fit(X_train_bagged,y_train_bagged.values.ravel())
            y_train_pred=model.predict(X_train)
            y_val_pred=model.predict(X_val)
        
        
            training_error_rate.append(mean_squared_error(y_train,y_train_pred))
            validation_error_rate.append(mean_squared_error(y_val,y_val_pred))
        
        
        
        train_mean_error_rate.append(np.array(training_error_rate).mean())
        train_std_error_rate.append(np.array(training_error_rate).std())
        validation_mean_error_rate.append(np.array(validation_error_rate).mean())
        validation_std_error_rate.append(np.array(validation_error_rate).std())
    
    
    x=np.arange(1,B+1)
    plt.plot(x,train_mean_error_rate,label='training data')
    plt.plot(x,validation_mean_error_rate,label='validation data')
    plt.ylabel('mean error rate')
    plt.xlabel('number of trees')
    plt.legend()
    plt.show()


    plt.plot(x,train_std_error_rate,label='training data')
    plt.plot(x,validation_std_error_rate,label='validation data')
    plt.ylabel('standard deviation of error rate')
    plt.xlabel('number of trees')
    plt.legend()
    plt.show()
    print('the lowest validation error is when B=',np.argmin(validation_mean_error_rate)+1,'with validation error of:',np.array(validation_mean_error_rate).min(),'with standar deviation of',validation_std_error_rate[np.argmin(validation_mean_error_rate)])
    

N=[0.1,0.2 ,0.3,0.4,0.5, 0.6,0.7,0.8, 0.9,0.999]
for j in N:
    find_vectors_and_plots(50,j,10,3)
    


#study the robusness to the noise for Random Forest
best_tree_nums=[]
validation_scores=[]
noise_std=np.arange(0.01,0.1,0.01)
for j in noise_std:
    valadation_scores={}
    
    for i in range(2,50):
        model=RandomForestRegressor(n_estimators=i,criterion='mse',max_depth=best_depth,max_features=None,random_state=0)
        X_train_std_noisy=X_train_std+np.random.normal(0,j,[5740,7])
        model.fit(X_train_std_noisy,y_train)
        y_val_pred=model.predict(X_val_std)
        valadation_scores.update({i+1:mean_squared_error(y_val,y_val_pred)})
    
    best_tree_num=min(valadation_scores, key=valadation_scores.get)
    best_tree_nums.append(best_tree_num)


    model=RandomForestRegressor(n_estimators=best_tree_num,criterion='mse',max_depth=best_depth,max_features=None,random_state=0)
    model.fit(X_train_std_noisy,y_train)
    y_train_pred=model.predict(X_train_std_noisy)
    validation_scores.append(mean_squared_error(y_val,model.predict(X_val_std)))
    
    


plt.plot(noise_std,best_tree_nums,c='r');plt.xlabel('noise_std');plt.ylabel('best_tree_number')
plt.show()
plt.plot(noise_std,validation_scores,c='r');plt.xlabel('noise_std');plt.ylabel('validation error')
plt.show()

In [ ]:
#ada_boost
valadation_scores={}
for i in np.arange(0.1, 3, 0.1):
    model=AdaBoostRegressor(DecisionTreeRegressor(max_depth=10,criterion='mse',
                                                  max_features=None,random_state=0),
                            n_estimators=48, learning_rate=i,random_state=0)
    model.fit(X_train,y_train)
    y_val_pred=model.predict(X_val)
    valadation_scores.update({i:mean_squared_error(y_val,y_val_pred)})
    
best_learning_rate=min(valadation_scores, key=valadation_scores.get)

model=AdaBoostRegressor(DecisionTreeRegressor(max_depth=10,criterion='mse',
                                              max_features=None,random_state=0),
                        n_estimators=48, learning_rate=0.8,random_state=0)
model.fit(X_train,y_train)
y_train_pred=model.predict(X_train)
y_val_pred=model.predict(X_val)
y_test_pred=model.predict(X_test)
print('Adaboost Train MSE is',mean_squared_error(y_train,y_train_pred))
print('Adaboost Valadation set MSE is:',mean_squared_error(y_val,y_val_pred))
print('best learning rate is:',best_learning_rate)
plt.plot(valadation_scores.keys(),valadation_scores.values())
plt.xlabel('Learning Rate')
plt.ylabel('Validation Error')
plt.show()


#study the variabity of diffrent training size for Ada boost
best_learning_rates=[]
validation_scores=[]
N=np.arange(0.01,0.1,0.01)
for j in N:
    valadation_scores={}
    
    for i in np.arange(0.1, 3, 0.1):
        model=AdaBoostRegressor(DecisionTreeRegressor(max_depth=10,criterion='mse',max_features=None,random_state=0),n_estimators=48, learning_rate=0.8,random_state=0)
        X_train_bagged,X_throw,y_train_bagged,y_throw=train_test_split(X_train,y_train,train_size=j ,random_state=0)
        model.fit(X_train_bagged,y_train_bagged)
        y_val_pred=model.predict(X_val)
        valadation_scores.update({i:mean_squared_error(y_val,y_val_pred)})
    
    best_learning_rate=min(valadation_scores, key=valadation_scores.get)
    best_learning_rates.append(best_learning_rate)


    model=AdaBoostRegressor(DecisionTreeRegressor(max_depth=best_depth,criterion='mse',max_features=None,random_state=0),n_estimators=best_tree_num, learning_rate=best_learning_rate,random_state=0)
    model.fit(X_train_bagged,y_train_bagged)
    y_train_pred=model.predict(X_train_bagged)
    validation_scores.append(mean_squared_error(y_val,model.predict(X_val)))
    test_scores.append(mean_squared_error(y_test,model.predict(X_test)))
    


plt.plot(N,best_learning_rates);plt.xlabel('Training Size');plt.ylabel('best_learning_rates')
plt.show()
plt.plot(N,validation_scores);plt.xlabel('Training Size');plt.ylabel('validation error')
plt.show()


#study the robusness to the noise for Ada boost
best_learning_rates=[]
validation_scores=[]
noise_std=np.arange(0.01,0.1,0.01)
for j in noise_std:
    valadation_scores={}
    
    for i in np.arange(0.1, 3, 0.1):
        model=AdaBoostRegressor(DecisionTreeRegressor(max_depth=10,criterion='mse',
                                                  max_features=None,random_state=0),
                            n_estimators=48, learning_rate=0.8,random_state=0)
        X_train_std_noisy=X_train_std+np.random.normal(0,j,[5740,7])
        model.fit(X_train_std_noisy,y_train)
        y_val_pred=model.predict(X_val_std)
        valadation_scores.update({i:mean_squared_error(y_val,y_val_pred)})
    
    best_learning_rate=min(valadation_scores, key=valadation_scores.get)
    best_learning_rates.append(best_learning_rate)


    model=AdaBoostRegressor(DecisionTreeRegressor(max_depth=10,criterion='mse',
                                                  max_features=None,random_state=0),
                            n_estimators=48, learning_rate=0.8,random_state=0)
    model.fit(X_train_std_noisy,y_train)
    y_train_pred=model.predict(X_train_std_noisy)
    validation_scores.append(mean_squared_error(y_val,model.predict(X_val_std)))
    test_scores.append(mean_squared_error(y_test,model.predict(X_test_std)))
    


plt.plot(noise_std,best_learning_rates,c='r');plt.xlabel('noise_std');plt.ylabel('best learning rate')
plt.show()
plt.plot(noise_std,validation_scores,c='r');plt.xlabel('noise_std');plt.ylabel('validation error')
plt.show()

In [ ]:
# chosen model
model=AdaBoostRegressor(DecisionTreeRegressor(max_depth=10,criterion='mse',
                                              max_features=None,random_state=0),
                        n_estimators=48, learning_rate=0.8,random_state=0)
model.fit(X_train,y_train)
y_train_pred=model.predict(X_train)
y_val_pred=model.predict(X_val)
y_test_pred=model.predict(X_test)
print('Adaboost Train MSE is',mean_squared_error(y_train,y_train_pred))
print('Adaboost Test MSE is:',mean_squared_error(y_test,y_test_pred))
print('R2 score error is:',1 - r2_score(y_test, y_test_pred))


#plot the regression functions
model.fit(X_train[['AT']], y_train.values.ravel())
predict_x = model.predict(X_train[['AT']])
predict_test = model.predict(X_test[['AT']])
X_grid = np.arange(min(np.array(X_train[['AT']])), max(np.array(X_train[['AT']])), 0.001)

X_grid = X_grid.reshape((len(X_grid), 1))

        
plt.scatter(X_train[['AT']], y_train, color='blue', label='training data points')
        
plt.plot(X_grid, model.predict(X_grid), color='green', label='regression function')
plt.title('Ada boost regression')
plt.xlabel('Ambient Temperature (AT)')
plt.ylabel('Elecatical Energy (PE)')
plt.legend(loc='best')
plt.show()

model.fit(X_train[['V']], y_train.values.ravel())
predict_x = model.predict(X_train[['V']])
predict_test = model.predict(X_test[['V']])
X_grid = np.arange(min(np.array(X_train[['V']])), max(np.array(X_train[['V']])), 0.001)

X_grid = X_grid.reshape((len(X_grid), 1))

        
plt.scatter(X_train[['V']], y_train, color='blue', label='training data points')
        
plt.plot(X_grid, model.predict(X_grid), color='green', label='regression function')
plt.title('Ada boost regression')
plt.xlabel('Exhaust Vacuum (V)')
plt.ylabel('Elecatical Energy (PE)')
plt.legend(loc='best')
plt.show()